Connected to dl_env (Python 3.9.16)

In [1]:
from X_nan_transformer import *
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn import set_config
set_config(transform_output="pandas")

class NaN_ColumnCreator_Total_Missing(BaseEstimator, TransformerMixin):
    """
    Die resultierenden Werte aus dem X_nan_transformer werden
    aufsummiert, so dass weniger Spalten benötigt werden,
    die NaN-Zeilen markieren.
    """
    def __init__(self):
        pass
    
    def filter_nans_total(self,X):
        try:
            nan_m_cols = X.filter(regex='(?<!NOT)_MISSING$')
            return nan_m_cols
        except:
            return X

    def fit(self, X, y=None):
        self.column_names_missing = X.filter(regex='(?<!NOT)_MISSING$').columns.tolist()
        return self
    
    def transform(self, X):
        try:
            X_nan_m_cols = self.filter_nans_total(X)
            print(X_nan_m_cols)
            X_transformed = X.copy()
            X_transformed['Total_Missing'] = X_nan_m_cols.sum(axis=1)
            X_transformed.drop(X_nan_m_cols.columns, axis=1, inplace=True)
            return X_transformed
        except Exception as e:
            print(f"Error: {e}")
            return X


    def get_feature_names_out(self, input_features=None):
            return self.column_names_missing



class NaN_ColumnCreator_Total_Not_Missing(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def filter_nans_total(self,X):
        try:
            nan_filter_total = X.filter(regex='NOT_MISSING$')
            return nan_filter_total
        except:
            X

    def fit(self, X, y=None):
        return self

    def transform(self, X):
            try:
                X_transformed = self.filter_nans_total()
                return X_transformed
            except:
                return X_transformed


    def get_feature_names_out(self, input_features=None):
        return self.columns_with_nan




def init_nan_marker_cat() -> Pipeline:
    return  Pipeline(
        steps=[
        ("nan_marker_cat", NaN_ColumnCreator_CatData()),
        ("one_hot_cat_nan", OneHotEncoder(handle_unknown="ignore",sparse=False)),
        # ("nan_marker_cat_total_missing", NaN_ColumnCreator_Total_Missing()), 
        # ("nan_marker_cat_total", ColumnTransformer(
        #     transformers=[
        #         ("nan_marker_cat_total_missing", NaN_ColumnCreator_Total_Missing(), make_column_selector(dtype_include=np.object_)), 
        #         # ("nan_marker_cat_total_not_missing", NaN_ColumnCreator_Total_Not_Missing())
        #     ]
        # ))
    ])


preprocessing = ColumnTransformer(
    [
#         ("cat", prep_cat, make_column_selector(dtype_include=np.object_)),
#         ("num", prep_num, make_column_selector(dtype_include=np.number)),
#         ("tukey", TukeyTransformer(), make_column_selector(dtype_include=np.number)),
#         ("z", Z_Transformer(threshold=3, z_scores_output=True), make_column_selector(dtype_include=np.number))
 
            
        # ("nan_marker_num", NaN_ColumnCreator_NumData(), make_column_selector(dtype_include=np.number)),
        ("nan_marker_cat", init_nan_marker_cat(), make_column_selector(dtype_include=np.object_))
        
    ],
     remainder='passthrough' # this will pass through any columns not specified in the transformers
)

test_data = pd.DataFrame({"Example_num_column":np.array([1,2,3,4,5,100000, np.nan]), 
                          "Example_cat_column":["Katze","Hund","Hund","Katze",np.nan, np.nan, np.nan],
                          "Example_cat_column1":["Katze","Katze","Hund","Hund",np.nan, np.nan, np.nan],
                         "Example_no_nan":np.array([1,2,3,4,5,100000, 500])})

preprocessed_data = pd.DataFrame(preprocessing.fit_transform(test_data), columns=preprocessing.get_feature_names_out())
preprocessed_data

c:\Users\jadelhelm\AppData\Local\anaconda3\envs\dl_env\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,nan_marker_cat__Example_cat_column_MISSING,nan_marker_cat__Example_cat_column_NOT_MISSING,nan_marker_cat__Example_cat_column1_MISSING,nan_marker_cat__Example_cat_column1_NOT_MISSING,remainder__Example_num_column,remainder__Example_no_nan
0,0.0,1.0,0.0,1.0,1.0,1
1,0.0,1.0,0.0,1.0,2.0,2
2,0.0,1.0,0.0,1.0,3.0,3
3,0.0,1.0,0.0,1.0,4.0,4
4,1.0,0.0,1.0,0.0,5.0,5
5,1.0,0.0,1.0,0.0,100000.0,100000
6,1.0,0.0,1.0,0.0,NaN,500


In [2]:
def filter_nans_total(X):
    try:
        nan_m_cols = X.filter(regex='(?<!NOT)_MISSING$')
        return nan_m_cols
    except:
        return X

In [3]:
columns_missing = filter_nans_total(preprocessed_data)
columns_missing

,nan_marker_cat__Example_cat_column_MISSING,nan_marker_cat__Example_cat_column1_MISSING
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,1.0,1.0
5,1.0,1.0
6,1.0,1.0


In [4]:
def transform(X):
    try:
        X_nan_m_cols = self.filter_nans_total(X)
        print(X_nan_m_cols)
        X_transformed = X.copy()
        X_transformed['Total_Missing'] = X_nan_m_cols.sum(axis=1)
        X_transformed.drop(X_nan_m_cols.columns, axis=1, inplace=True)
        return X_transformed
    except Exception as e:
        print(f"Error: {e}")
        return X

In [1]:
import pandas as pd

# Create a sample DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

# Convert each element to string
df_str = df.applymap(str)

# Concatenate strings in each row
df_str['concatenated'] = df_str.agg(''.join, axis=1)

print(df_str['concatenated'])


0    147
1    258
2    369
Name: concatenated, dtype: object
